In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender 
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [6]:
import optuna as op

In [7]:
# URM_train = sps.load_npz('URM_train.npz')
# URM_val = sps.load_npz('URM_val.npz')

# URM_train_val, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
# URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_train_val, train_percentage = 0.85)

In [8]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [9]:
print(f"Train size: {URM_train.getnnz()}, Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

Train size: 1274929, Val size: 224987, Test size: 264691


In [10]:
evaluator_val = EvaluatorHoldout(URM_val, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 527 ( 1.5%) Users that have less than 1 test interactions


In [17]:
def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

In [12]:
# We pass as paramether the recommender class

def evaluate_algorithm(URM_test, recommender_object, at=10):
    
    #cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, cutoff=at)
            num_eval+=1

            #cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    #cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    return MAP, cumulative_recall


In [13]:
slim_model_train = SLIMElasticNetRecommender(URM_train)
#slim_model_train.fit(topK =320, l1_ratio=0.1527515113685262, alpha=0.00022576119312621578)
slim_model_train.load_model('result_experiments/SLIM/', 'slim_320_train_ens')

#slim_model_train.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)

#slim_model_train.load_model('result_experiments/SLIM/', 'slim_307_train_ens')
# slim_model_all = SLIMElasticNetRecommender(URM_train)
# slim_model_all.load_model('result_experiments/SLIM/', 'slim_300_2_weights')

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_320_train_ens'
SLIMElasticNetRecommender: Loading complete


In [27]:
SLIM_ElasticRecall_eval = SLIMElasticNetRecommender(URM_train)
#SLIM_ElasticRecall_eval.fit(topK= 296, l1_ratio=0.10442943478590742, alpha=0.00035921608889258387)
SLIM_ElasticRecall_eval.load_model("result_experiments/SLIM/", "SLIM_ElasticRecall_eval")

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/SLIM_ElasticRecall_eval'
SLIMElasticNetRecommender: Loading complete


In [28]:
slim_recall_train_res, _ = evaluator_val.evaluateRecommender(SLIM_ElasticRecall_eval)

EvaluatorHoldout: Processed 34728 (100.0%) in 13.21 sec. Users per second: 2629


In [29]:
slim_recall_train_res["MAP"], slim_recall_train_res["RECALL"], slim_recall_train_res["PRECISION"]

(cutoff
 10    0.030483
 Name: MAP, dtype: object,
 cutoff
 10    0.128475
 Name: RECALL, dtype: object,
 cutoff
 10    0.065403
 Name: PRECISION, dtype: object)

In [14]:
slim_train_res, _ = evaluator_val.evaluateRecommender(slim_model_train)

EvaluatorHoldout: Processed 34728 (100.0%) in 13.28 sec. Users per second: 2615


In [15]:
slim_train_res["MAP"], slim_train_res["RECALL"], slim_train_res["PRECISION"]

(cutoff
 10    0.030645
 Name: MAP, dtype: object,
 cutoff
 10    0.127663
 Name: RECALL, dtype: object,
 cutoff
 10    0.065405
 Name: PRECISION, dtype: object)

In [18]:
evaluate_algorithm(URM_val, slim_model_train)

(0.06301192406753578, 0.12766340864666403)

In [19]:
slim_model_train_val = SLIMElasticNetRecommender(URM_train_val)
#slim_model_train_val.fit(topK=320, l1_ratio=0.1527515113685262, alpha=0.00022576119312621578)
slim_model_train_val.load_model('result_experiments/SLIM/', 'slim_320_train_val_ens')

#slim_model_train_val.load_model('result_experiments/SLIM/', 'slim_307_train_val_ens')
#slim_model_train_val.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)

SLIMElasticNetRecommender: Loading model from file 'result_experiments/SLIM/slim_320_train_val_ens'
SLIMElasticNetRecommender: Loading complete


In [20]:
slim_train_val_res, _ = evaluator_test.evaluateRecommender(slim_model_train_val)

EvaluatorHoldout: Processed 35209 (100.0%) in 14.48 sec. Users per second: 2432


In [21]:
slim_train_val_res["MAP"], slim_train_val_res["RECALL"], slim_train_val_res["PRECISION"]

(cutoff
 10    0.047761
 Name: MAP, dtype: object,
 cutoff
 10    0.148651
 Name: RECALL, dtype: object,
 cutoff
 10    0.090886
 Name: PRECISION, dtype: object)

In [22]:
evaluate_algorithm(URM_test, slim_model_train_val)

(0.08371722317381694, 0.14865124595112233)

In [23]:
rp3_beta_train = RP3betaRecommender(URM_train)
#rp3_beta_train.load_model('result_experiments/RP3beta/', 'rp3_beta_12_weights_train_ens')
#rp3_beta_train.fit(alpha=0.34989902568351894, beta=0.1817338725671425, topK=12, min_rating=0.0, implicit=False, normalize_similarity=True)

#rp3_beta_train.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)
rp3_beta_train.load_model('result_experiments/RP3beta/', 'rp3_beta_19_weights_train_ens')

rp3_beta_train_val = RP3betaRecommender(URM_train_val)
#rp3_beta_train_val.load_model('result_experiments/RP3beta/', 'rp3_beta_12_weights_train_val_ens')

#rp3_beta_train_val.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)
rp3_beta_train_val.load_model('result_experiments/RP3beta/', 'rp3_beta_19_weights_train_val_ens')


RP3betaRecommender: Loading model from file 'result_experiments/RP3beta/rp3_beta_19_weights_train_ens'
RP3betaRecommender: Loading complete
RP3betaRecommender: Loading model from file 'result_experiments/RP3beta/rp3_beta_19_weights_train_val_ens'
RP3betaRecommender: Loading complete


In [24]:
evaluate_algorithm(URM_val, rp3_beta_train), evaluate_algorithm(URM_test, rp3_beta_train_val)

((0.058588907885092474, 0.12086354687120603),
 (0.07551240567510359, 0.13757912073177564))

In [25]:
rp3_beta_train_res, _ = evaluator_val.evaluateRecommender(rp3_beta_train)
rp3_beta_train_val_res, _ = evaluator_test.evaluateRecommender(rp3_beta_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 8.90 sec. Users per second: 3903
EvaluatorHoldout: Processed 35209 (100.0%) in 9.13 sec. Users per second: 3857


In [26]:
rp3_beta_train_res["MAP"], rp3_beta_train_res["RECALL"], rp3_beta_train_val_res["MAP"], rp3_beta_train_val_res["RECALL"]

(cutoff
 10    0.02844
 Name: MAP, dtype: object,
 cutoff
 10    0.120864
 Name: RECALL, dtype: object,
 cutoff
 10    0.042595
 Name: MAP, dtype: object,
 cutoff
 10    0.137579
 Name: RECALL, dtype: object)

In [26]:
# similarity='jaccard', topK=5, shrink=12, normalize=False

knn_train = ItemKNNCFRecommender(URM_train)
knn_train.fit(
    similarity='tversky', 
    topK = 6,
    shrink= 21.589349496863413,
    feature_weighting= 'TF-IDF',
    normalize= False,
    tversky_alpha= 0.16725184866550824,
    tversky_beta= 1.722012798691049
)

#knn_train.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_ens.zip")
# knn_train.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

knn_train_val = ItemKNNCFRecommender(URM_train_val)
knn_train_val.fit(
    similarity='tversky', 
    topK = 6,
    shrink= 21.589349496863413,
    feature_weighting= 'TF-IDF',
    normalize= False,
    tversky_alpha= 0.16725184866550824,
    tversky_beta= 1.722012798691049
)
#knn_train_val.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_train_val_ens.zip")
# knn_train_val.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

Similarity column 38121 (100.0%), 4877.97 column/sec. Elapsed time 7.81 sec
Similarity column 38121 (100.0%), 4532.06 column/sec. Elapsed time 8.41 sec


In [27]:
evaluate_algorithm(URM_val, knn_train), evaluate_algorithm(URM_test, knn_train_val)

(0.058090943997984236, 0.0747781539320609)

In [28]:
knn_train_res, _ = evaluator_val.evaluateRecommender(knn_train)
knn_train_val_res, _ = evaluator_test.evaluateRecommender(knn_train_val)

EvaluatorHoldout: Processed 34728 (100.0%) in 8.91 sec. Users per second: 3898
EvaluatorHoldout: Processed 35209 (100.0%) in 9.26 sec. Users per second: 3804


In [29]:
knn_train_res["MAP"], knn_train_res["RECALL"], knn_train_val_res["MAP"], knn_train_val_res["RECALL"]

(cutoff
 10    0.02813
 Name: MAP, dtype: object,
 cutoff
 10    0.118551
 Name: RECALL, dtype: object,
 cutoff
 10    0.04233
 Name: MAP, dtype: object,
 cutoff
 10    0.135404
 Name: RECALL, dtype: object)

# Optuna

In [30]:
results_dict={
    'Iteration':[],
    'SLIM_w':[],
    'RP3b_w':[],
    'ItemKNN_w':[],
    'MAP_val':[],
    'MAP_test':[],
}

In [31]:
new_best_on_test={
    'SLIM_w':0,
    'RP3b_w':0,
    'ItemKNN_w':0,
    'MAP_test':0
}

In [32]:
improving_treshold=0.70 

In [33]:
import optuna as op

best_on_val = 0.03

def objective_function_weight_hybrid(trial):
    results_dict['Iteration'].append(trial.number)
    n = 3
    x = []
    for i in range(n):
        x.append(- np.log(trial.suggest_float(f"x_{i}", 0, 1)))
    weights = []
    for i in range(n):
        weights.append(x[i] / sum(x))
        if i==0:
            key='SLIM_w'
        elif i==1:
            key='RP3b_w'
        else:
            key='ItemKNN_w'
        results_dict[key].append(weights[i])

    for i in range(n):
        trial.set_user_attr(f"weights_{i}",weights[i])
        
    #PRINT WEIGHTS CHOSEN ###################################################
    print("__________Iteration " + str(trial.number) + "______________")
    print("SLIM weight: "+str(weights[0]))
    print("RP3b weight: "+str(weights[1]))
    print("ItemKNN weight: "+str(weights[2]))##
    
    #BUILD HYBRID ###########################################################
    new_similarity_train = weights[0] * slim_model_train.W_sparse + weights[1] * rp3_beta_train.W_sparse + weights[2] * knn_train.W_sparse
    recommender_object_train = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_object_train.fit(new_similarity_train)
    res_on_val, _ = evaluator_val.evaluateRecommender(recommender_object_train)
    results_dict['MAP_val'].append(res_on_val['MAP'][10])

    #print("VAL", res_on_val['MAP'][10], best_on_val * improving_treshold)
    if res_on_val['MAP'][10] >= best_on_val * improving_treshold:
        new_similarity_train_val = weights[0] * slim_model_train_val.W_sparse + weights[1] * rp3_beta_train_val.W_sparse + weights[2] * knn_train_val.W_sparse
        recommender_object_train_val = ItemKNNCustomSimilarityRecommender(URM_train_val)
        recommender_object_train_val.fit(new_similarity_train_val)
        res_on_test, _ = evaluator_test.evaluateRecommender(recommender_object_train_val)
        results_dict['MAP_test'].append(res_on_test["MAP"][10])

        #print("Test", res_on_test["MAP"][10], new_best_on_test['MAP_test'])
        if res_on_test["MAP"][10] > new_best_on_test['MAP_test']:
            new_best_on_test['SLIM_w']=weights[0]
            new_best_on_test['RP3b_w']=weights[1]
            new_best_on_test['ItemKNN_w']=weights[2]
            new_best_on_test['MAP_test']=res_on_test["MAP"][10]
            print("New best on test found! Score is "+ str(res_on_test["MAP"][10]))
            print(f"weights: SLIM_w: {weights[0]}, RP3b_w: {weights[1]}, ItemKNN_w: {weights[2]}")
            
        
        
    else:
         results_dict['MAP_test'].append(0.0)
    return res_on_val['MAP'][10]

In [34]:
study = op.create_study(direction="maximize")
study.optimize(objective_function_weight_hybrid,
                      callbacks=[],
                      n_trials = 500)

[I 2024-12-27 01:07:26,998] A new study created in memory with name: no-name-d29ea8de-e660-4063-8ab5-08e4a5cf9d77


__________Iteration 0______________
SLIM weight: 0.008681939735686655
RP3b weight: 0.0057793576347350524
ItemKNN weight: 0.9855387026295783
EvaluatorHoldout: Processed 34728 (100.0%) in 13.18 sec. Users per second: 2635
EvaluatorHoldout: Processed 35209 (100.0%) in 14.35 sec. Users per second: 2454


[I 2024-12-27 01:07:54,642] Trial 0 finished with value: 0.02817954406672374 and parameters: {'x_0': 0.9862799545135115, 'x_1': 0.9908458216601674, 'x_2': 0.20841438528577194}. Best is trial 0 with value: 0.02817954406672374.


New best on test found! Score is 0.04243987152454994
weights: SLIM_w: 0.008681939735686655, RP3b_w: 0.0057793576347350524, ItemKNN_w: 0.9855387026295783
__________Iteration 1______________
SLIM weight: 0.45714239921586314
RP3b weight: 0.08279925818436454
ItemKNN weight: 0.4600583425997722
EvaluatorHoldout: Processed 34728 (100.0%) in 13.33 sec. Users per second: 2604
EvaluatorHoldout: Processed 35209 (100.0%) in 14.50 sec. Users per second: 2428


[I 2024-12-27 01:08:22,567] Trial 1 finished with value: 0.030270789226386118 and parameters: {'x_0': 0.3854972696330816, 'x_1': 0.8414313827156279, 'x_2': 0.3831604621807765}. Best is trial 1 with value: 0.030270789226386118.


...

[I 2024-12-27 04:53:03,633] Trial 497 finished with value: 0.031077667533264255 and parameters: {'x_0': 0.03738423277027156, 'x_1': 0.28134148679313303, 'x_2': 0.7919823380248096}. Best is trial 358 with value: 0.031101669253663686.


__________Iteration 498______________
SLIM weight: 0.7057232212523773
RP3b weight: 0.24660066220809615
ItemKNN weight: 0.04767611653952661
EvaluatorHoldout: Processed 34728 (100.0%) in 13.00 sec. Users per second: 2670
EvaluatorHoldout: Processed 35209 (100.0%) in 14.03 sec. Users per second: 2510


[I 2024-12-27 04:53:30,762] Trial 498 finished with value: 0.03105894264908499 and parameters: {'x_0': 0.018861729492764816, 'x_1': 0.24971058022689846, 'x_2': 0.7647236189917076}. Best is trial 358 with value: 0.031101669253663686.


__________Iteration 499______________
SLIM weight: 0.6412992824139915
RP3b weight: 0.32067435287959795
ItemKNN weight: 0.03802636470641056
EvaluatorHoldout: Processed 34728 (100.0%) in 12.98 sec. Users per second: 2676
EvaluatorHoldout: Processed 35209 (100.0%) in 14.02 sec. Users per second: 2512


[I 2024-12-27 04:53:57,848] Trial 499 finished with value: 0.031066108313861272 and parameters: {'x_0': 0.04834837899154544, 'x_1': 0.21985698256251174, 'x_2': 0.8355825299520795}. Best is trial 358 with value: 0.031101669253663686.


In [35]:
results_df=pd.DataFrame(results_dict)
results_df.to_csv("result_experiments/ensemble_SLIM_RP3_KNNCFR/results_prof_eval_27_12_24.csv",sep=',',index=False)

In [60]:
similarity_best = 0.862708291473966 * slim_model_train_val.W_sparse + 0.10238146945917358 * rp3_beta_train_val.W_sparse + 0.034910239066860506 * knn_train_val.W_sparse
recommender_best = ItemKNNCustomSimilarityRecommender(URM_train_val)
recommender_best.fit(similarity_best)

In [62]:
recommender_best_res, _ = evaluator_test.evaluateRecommender(recommender_best)
recommender_best_res["MAP"], recommender_best_res["RECALL"], recommender_best_res["PRECISION"] 

EvaluatorHoldout: Processed 35209 (100.0%) in 14.67 sec. Users per second: 2400


(cutoff
 10    0.04793
 Name: MAP, dtype: object,
 cutoff
 10    0.149136
 Name: RECALL, dtype: object,
 cutoff
 10    0.09109
 Name: PRECISION, dtype: object)

In [64]:
evaluate_algorithm(URM_test, recommender_best)

0.08402824403040553

In [63]:
results_df.sort_values('MAP_test', ascending=False)

,Iteration,SLIM_w,RP3b_w,ItemKNN_w,MAP_val,MAP_test
282,282,0.754314,0.222048,0.023638,0.030990,0.048032
33,33,0.753455,0.230423,0.016122,0.030994,0.048030
262,262,0.718897,0.251402,0.029701,0.031051,0.048029
225,225,0.756138,0.221332,0.022530,0.030992,0.048028
384,384,0.727622,0.241647,0.030731,0.031037,0.048028
...,...,...,...,...,...,...
473,473,0.016375,0.822663,0.160961,0.029469,0.044360
266,266,0.044960,0.845908,0.109132,0.029373,0.044305
30,30,0.143846,0.150658,0.705496,0.029306,0.044252
155,155,0.017493,0.862692,0.119815,0.029265,0.044074


In [67]:
results_df=pd.DataFrame(results_dict)
results_df.to_csv("result_experiments/ensemble_SLIM_RP3_KNNCFR/results_prof_eval_18_12_24.csv",sep=',',index=False)

In [ ]:
res_best, _ = evaluator_test.evaluateRecommender(recommender_best)

In [ ]:
res_best

In [35]:
results_best[results_best["MAP_test"]==results_best["MAP_test"].max()]

,Iteration,SLIM_w,RP3b_w,ItemKNN_w,MAP_val,MAP_test
148,148,0.867298,0.105076,0.027626,0.030877,0.047923


In [43]:
new_best_on_test

{'SLIM_w': 0.7543139107121462,
 'RP3b_w': 0.2220484447741522,
 'ItemKNN_w': 0.023637644513701594,
 'MAP_test': 0.04803199330798644}

In [57]:
# slim_model_train.save_model("result_experiments/SLIM/", "slim_320_train_ens")
# slim_model_train_val.save_model("result_experiments/SLIM/", "slim_320_train_val_ens")

SLIMElasticNetRecommender: Saving model in file 'result_experiments/SLIM/slim_320_train_ens'
SLIMElasticNetRecommender: Saving complete
SLIMElasticNetRecommender: Saving model in file 'result_experiments/SLIM/slim_320_train_val_ens'
SLIMElasticNetRecommender: Saving complete


In [58]:
# knn_train.save_model("result_experiments/ItemKNNCFRecommender/", "knn_6_train_ens")
# knn_train_val.save_model("result_experiments/ItemKNNCFRecommender/", "knn_6_train_val_ens")

ItemKNNCFRecommender: Saving model in file 'result_experiments/ItemKNNCFRecommender/knn_6_train_ens'
ItemKNNCFRecommender: Saving complete
ItemKNNCFRecommender: Saving model in file 'result_experiments/ItemKNNCFRecommender/knn_6_train_val_ens'
ItemKNNCFRecommender: Saving complete


In [59]:
# rp3_beta_train.save_model("result_experiments/RP3beta/", "rp3_beta_19_weights_train_ens")
# rp3_beta_train_val.save_model("result_experiments/RP3beta/", "rp3_beta_19_weights_train_val_ens")

RP3betaRecommender: Saving model in file 'result_experiments/RP3beta/rp3_beta_19_weights_train'
RP3betaRecommender: Saving complete
RP3betaRecommender: Saving model in file 'result_experiments/RP3beta/rp3_beta_19_weights_train_val'
RP3betaRecommender: Saving complete


In [57]:
# results_df=pd.DataFrame(results_dict)
# results_df.to_csv("result_experiments/ensemble_SLIM_RP3_KNNCFR/results_prof_eval_11_12_24.csv",sep=',',index=False)

In [63]:
# sps.save_npz('URM_train_ens.npz', URM_train)
# sps.save_npz('URM_train_val_ens.npz', URM_train_val)
# sps.save_npz('URM_test_ens.npz', URM_test)

In [70]:
print(new_best_on_test['SLIM_w'])
print(new_best_on_test['RP3b_w'])
print(new_best_on_test['ItemKNN_w'])

0.8011551243595696
0.176441040480307
0.022403835160123328


In [49]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [38]:
SLIM_all = SLIMElasticNetRecommender(URM_all)
#SLIM_all.fit(l1_ratio=0.11006885790633625, alpha=0.0002551115306127753, topK = 307)
SLIM_all.fit(topK =320, l1_ratio=0.1527515113685262, alpha=0.00022576119312621578)
#SLIM_all.load_model("result_experiments/SLIM/", "slim_307_weights_all")


SLIMElasticNetRecommender: Processed 6617 (17.4%) in 5.00 min. Items per second: 22.05
SLIMElasticNetRecommender: Processed 11348 (29.8%) in 21.42 min. Items per second: 8.83
SLIMElasticNetRecommender: Processed 18122 (47.5%) in 26.42 min. Items per second: 11.43
SLIMElasticNetRecommender: Processed 24984 (65.5%) in 31.42 min. Items per second: 13.25
SLIMElasticNetRecommender: Processed 31711 (83.2%) in 36.42 min. Items per second: 14.51
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 41.17 min. Items per second: 15.43


In [39]:
RP3_beta_all = RP3betaRecommender(URM_all)
RP3_beta_all.fit(alpha=0.4176290154380183, beta=0.21520987912346945, topK=19, min_rating=0.0, implicit=False, normalize_similarity=True)

#RP3_beta_all.load_model("result_experiments/RP3beta/", "rp3_beta_12_weights_all")


RP3betaRecommender: Similarity column 38121 (100.0%), 3795.54 column/sec. Elapsed time 10.04 sec


In [40]:
KNN_all = ItemKNNCFRecommender(URM_all)
KNN_all.fit(
    similarity='tversky', 
    topK = 6,
    shrink= 21.589349496863413,
    feature_weighting= 'TF-IDF',
    normalize= False,
    tversky_alpha= 0.16725184866550824,
    tversky_beta= 1.722012798691049
)
# KNN_all.fit(similarity='tversky', topK=4, shrink=18,
#                   feature_weighting='none', tversky_alpha=0.1263621,
#                   tversky_beta=1.72181042,
#                   normalize=False)

KNN_all.load_model("result_experiments/ItemKNNCFRecommender/", "knn_4_weights_all")

Similarity column 38121 (100.0%), 4254.06 column/sec. Elapsed time 8.96 sec
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender/knn_4_weights_all'
ItemKNNCFRecommender: Loading complete


In [53]:
# iter 239 0.04801274656744886
# 0.7380796311636464 * SLIM_all.W_sparse + 0.21602197286950797 * RP3_beta_all.W_sparse + 0.04589839596684567 * KNN_all.W_sparse

similarity_all = 0.862708291473966 * SLIM_all.W_sparse + 0.10238146945917358 * RP3_beta_all.W_sparse + 0.034910239066860506 * KNN_all.W_sparse
recommender_best = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_best.fit(similarity_all)

In [54]:
recommender_best_res, _ = evaluator_test.evaluateRecommender(recommender_best)

EvaluatorHoldout: Processed 35209 (100.0%) in 15.08 sec. Users per second: 2335


In [55]:
recommender_best_res

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.985253,0.0,0.985253,0.258398,13.564324,0.999893,0.422648,0.918636,1.281876,0.196381


In [56]:
write_submission_lib_model(recommender_best, "ensemble_SLIM_RP3beta_KNN_11.csv")

In [72]:
recommender_best.save_model("result_experiments/ensemble_SLIM_RP3_KNNCFR/", "ensemble_weights_11_12_24")

ItemKNNCustomSimilarityRecommender: Saving model in file 'result_experiments/ensemble_SLIM_RP3_KNNCFR/ensemble_weights_11_12_24'
ItemKNNCustomSimilarityRecommender: Saving complete


In [73]:
SLIM_all.save_model("result_experiments/SLIM/", "slim_307_weights_all")

SLIMElasticNetRecommender: Saving model in file 'result_experiments/SLIM/slim_307_weights_all'
SLIMElasticNetRecommender: Saving complete


In [74]:
RP3_beta_all.save_model("result_experiments/RP3beta/", "rp3_beta_12_weights_all")

RP3betaRecommender: Saving model in file 'result_experiments/RP3beta/rp3_beta_12_weights_all'
RP3betaRecommender: Saving complete


In [75]:
KNN_all.save_model("result_experiments/ItemKNNCFRecommender/", "knn_4_weights_all")

ItemKNNCFRecommender: Saving model in file 'result_experiments/ItemKNNCFRecommender/knn_4_weights_all'
ItemKNNCFRecommender: Saving complete
